In [1]:
import pandas as pd
import requests as req
import json
import time,os
from dotenv import load_dotenv
from datetime import datetime
load_dotenv()
header = {"X-API-Key" : os.getenv('API_key')}

In [2]:
df = pd.read_csv('loc_sen.csv')
df.head()

,Location_id,Sensor_id,State,Area,Sensor_name,Latitude,Longitude
0,358456,2028433,Andhra Pradesh,Gulzarpet,co µg/m³,14.675886,77.593027
1,358456,2028399,Andhra Pradesh,Gulzarpet,no2 µg/m³,14.675886,77.593027
2,358456,2028438,Andhra Pradesh,Gulzarpet,pm10 µg/m³,14.675886,77.593027
3,358456,12237339,Andhra Pradesh,Gulzarpet,pm10 µg/m³,14.675886,77.593027
4,358456,2028432,Andhra Pradesh,Gulzarpet,pm25 µg/m³,14.675886,77.593027


In [7]:
sensors_ids = df['Sensor_id'].sort_values()
sensors_ids

2910          23
443           35
439           36
452          169
459          381
          ...   
1985    12708245
1986    12708246
1678    12815121
1679    12815122
1004    12961441
Name: Sensor_id, Length: 3042, dtype: int64

In [11]:
start_time = time.time()
index = 0
track = 0
end_time = time.time()
sensors_in_range = []
elapsed_time = end_time - start_time
df_inrange = pd.DataFrame(columns=[df.columns])
df_inrange.to_csv('location_sensors.csv',mode= 'w', index=False)
for sensor in sensors_ids:
    try:
        print(f'{index} ', end = '')
        r = req.get(url= f'https://api.openaq.org/v3/sensors/{sensor}?limit=100&page=1',headers=header)
        if r.status_code == 200:
            request = r.json()['results']
            try:
                year_start = int(request[0]['coverage']['datetimeFrom']['local'].split('-')[0])
                year_end = int(request[0]['coverage']['datetimeTo']['local'].split('-')[0])
                # print(year_start, year_end)
                if year_start<=2020 and year_end>= 2021:
                        sensors_in_range.append(sensor)
                        df_inrange = df[df['Sensor_id'] == sensor]
                        df_inrange = df_inrange.reset_index(drop=True)
                        df_inrange.to_csv('location_sensors.csv', index=False,header = False, mode = 'a')
            except:
                print('Error on fetching data!!')
        else :
            print(f"Sensor : {sensor} invalid error code : {r.status_code}")
    except Exception as e:
        print("error" , e)        
    index+=1
    track += 1
    end_time = time.time()
    elapsed_time = end_time - start_time
    if track == 60 and elapsed_time < 60 :
        print('Wait for :',61-elapsed_time)
        time.sleep(round(1))
        track = 0
        start_time = time.time()

0 Error on fetching data!!
1 2 3 Error on fetching data!!
4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 Wait for : 30.443922519683838
60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 Wait for : 28.603332996368408
120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 Wait for : 30.321179151535034
180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 Wait for :

In [ ]:
len(sensors_in_range)


1153

In [13]:
sensors_in_range = pd.read_csv('location_sensors.csv')
sensors_in_range = sensors_in_range['Sensor_id']

In [14]:
df_inrange = (df[df['Sensor_id'].isin(sensors_in_range)]).reset_index(drop=True)
df_inrange.head()

,Location_id,Sensor_id,State,Area,Sensor_name,Latitude,Longitude
0,5628,14969,Andhra Pradesh,GVM Corporation,co µg/m³,17.722682,83.308197
1,5628,14957,Andhra Pradesh,GVM Corporation,no2 µg/m³,17.722682,83.308197
2,5628,14947,Andhra Pradesh,GVM Corporation,pm10 µg/m³,17.722682,83.308197
3,5628,14944,Andhra Pradesh,GVM Corporation,pm25 µg/m³,17.722682,83.308197
4,5628,14939,Andhra Pradesh,GVM Corporation,so2 µg/m³,17.722682,83.308197


In [16]:
header = {"X-API-Key" : os.getenv("API_KEY")}
dataset = []
dataset_df = pd.DataFrame(dataset, columns=['Sensor_id', 'Value', 'DateTimeFrom', 'DateTimeTo'])
dataset_df.to_csv('measurements.csv',index=False)

for sensor in sensors_in_range:
    dataset = []
    page = 1
    while True:
        try:
            request = req.get(f'https://api.openaq.org/v3/sensors/{sensor}/days/monthly?limit=1000&page={page}', headers = header)
            raw = request.json()
            data = raw['results']

            if type(raw['meta']['found']) == int :
                try:
                    for param in data:
                        if '2020' in param['period']['datetimeFrom']['local']:
                            dataset.append([sensor, param['value'], param['period']['datetimeFrom']['local'].split("T")[0], param['period']['datetimeTo']['local'].split('T')[0]])
                    print(f'Data Collected from : {sensor}')
                    break
                except:
                    continue
            else:
                for param in data:
                    try:
                        if '2020' in param['period']['datetimeFrom']['local']:
                            dataset.append([sensor, param['value'], param['period']['datetimeFrom']['local'].split("T")[0], param['period']['datetimeTo']['local'].split('T')[0]])
                    except:
                        continue
                page+=1
                    
        except Exception as e:
            print(f'error in fetching from sensor , {sensor}, Error : , {e}')
    sensor_df = pd.DataFrame(dataset, columns=['Sensor_id', 'Value', 'DateTimeFrom', 'DateTimeTo'])    
    sensor_df.to_csv('measurements.csv', mode="a",index=False, header=False)
    time.sleep(1)

Data Collected from : 710
Data Collected from : 712
Data Collected from : 713
Data Collected from : 714
Data Collected from : 715
Data Collected from : 1166
Data Collected from : 1205
Data Collected from : 1207
Data Collected from : 1208
Data Collected from : 1209
Data Collected from : 1545
Data Collected from : 1546
Data Collected from : 4712
Data Collected from : 5231
Data Collected from : 5339
Data Collected from : 5340
Data Collected from : 5343
Data Collected from : 5345
Data Collected from : 5357
Data Collected from : 5359
Data Collected from : 5396
Data Collected from : 5397
Data Collected from : 5399
Data Collected from : 5402
Data Collected from : 5403
Data Collected from : 5405
Data Collected from : 5406
Data Collected from : 5409
Data Collected from : 14237
Data Collected from : 14238
Data Collected from : 14311
Data Collected from : 14319
Data Collected from : 14322
Data Collected from : 14519
Data Collected from : 14520
Data Collected from : 14521
Data Collected from : 145

In [ ]:
(df_inrange.head(2)).merge(dataset_df, on='Sensor_id')

Location_id     0
Sensor_id       0
Area            0
City            0
Sensor_name     0
Latitude        0
Longitude       0
Value           0
DateTimeFrom    0
DateTimeTo      0
dtype: int64

In [2]:
import pandas as pd
import requests as req
import json
import time,os
from dotenv import load_dotenv
from datetime import datetime
load_dotenv()
header = {"X-API-Key" : os.getenv('API_key')}

In [3]:
measurements = pd.read_csv('measurements.csv')
sensors = pd.read_csv('location_sensors.csv')


In [6]:
measurements.head()

,Sensor_id,Value,DateTimeFrom,DateTimeTo
0,710,881.0,2020-01-01,2020-02-01
1,710,912.0,2020-02-01,2020-03-01
2,710,735.0,2020-03-01,2020-04-01
3,710,495.0,2020-04-01,2020-05-01
4,710,579.0,2020-05-01,2020-06-01


In [12]:
(measurements['Sensor_id'] == 710).sum()

np.int64(12)

In [13]:
sensors["Percent_Coverage"] = [round((measurements['Sensor_id'] == x).sum()/12 * 100, 2) for x in sensors['Sensor_id']]
sensors.shape

(1153, 8)

In [14]:
sensors

,Location_id,Sensor_id,State,Area,Sensor_name,Latitude,Longitude,Percent_Coverage
0,407,710,Telangana,Zoo Park,co µg/m³,17.349694,78.451437,100.00
1,407,712,Telangana,Zoo Park,no2 µg/m³,17.349694,78.451437,100.00
2,407,713,Telangana,Zoo Park,so2 µg/m³,17.349694,78.451437,100.00
3,407,714,Telangana,Zoo Park,pm25 µg/m³,17.349694,78.451437,100.00
4,407,715,Telangana,Zoo Park,pm10 µg/m³,17.349694,78.451437,91.67
...,...,...,...,...,...,...,...,...
1148,42240,227037,Assam,Pan Bazaar,pm10 µg/m³,26.187500,91.744194,8.33
1149,42240,227038,Assam,Pan Bazaar,co µg/m³,26.187500,91.744194,8.33
1150,42240,227039,Assam,Pan Bazaar,no2 µg/m³,26.187500,91.744194,8.33
1151,5615,234003,Chhattisgarh,Sidhu Kanhu Indoor Stadium,pm25 µg/m³,23.540435,87.289222,8.33


In [15]:
high_coverage_sensors = sensors[sensors['Percent_Coverage'] >= 30].reset_index(drop=True)

In [18]:
high_coverage_sensors.shape

(986, 8)

In [19]:
loc_meas = high_coverage_sensors.merge(measurements)

In [20]:
loc_meas[['Sensor_name', 'Value']].groupby('Sensor_name').describe()

Value                                                        \
              count        mean         std    min     25%    50%     75%   
Sensor_name                                                                 
co µg/m³     2289.0  842.054865  582.507413 -150.0  463.00  717.0  1100.0   
no2 µg/m³    2340.0   26.998158   22.855726    0.0   11.70   20.0    34.6   
pm10 µg/m³   2115.0  123.166492  125.498910    0.0   56.10   95.2   153.0   
pm25 µg/m³   2373.0   64.081987   66.791124  -23.8   26.10   44.1    75.5   
so2 µg/m³    2244.0   51.715409  840.549650    0.0    6.72   10.7    17.5   

                      
                 max  
Sensor_name           
co µg/m³      4690.0  
no2 µg/m³      176.0  
pm10 µg/m³    4010.0  
pm25 µg/m³     985.0  
so2 µg/m³    34400.0

In [21]:
loc_meas.describe()

,Location_id,Sensor_id,Latitude,Longitude,Percent_Coverage,Value
count,11361.000000,11361.000000,11361.000000,11361.000000,11361.000000,11361.000000
mean,7374.455418,22811.310976,23.796927,78.287281,97.261837,221.745651
std,2157.670599,8846.362595,5.959831,4.150266,8.286610,556.659217
min,301.000000,710.000000,8.514909,72.591912,33.330000,-150.000000
25%,5633.000000,15110.000000,19.252920,76.302765,100.000000,15.900000
50%,6954.000000,20062.000000,25.771061,77.241060,100.000000,43.500000
75%,8365.000000,34789.000000,28.646233,78.433215,100.000000,159.000000
max,10921.000000,36703.000000,31.620000,94.098988,100.000000,34400.000000


In [23]:
loc_meas.to_csv('measure.csv',index=False)